In [4]:
%matplotlib notebook

# Obviously
import pandas as pd
import numpy as np

# Plotting set-up
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc
# Command to reset plot styles to default: mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('seaborn-poster')
mpl.rcParams['font.family'] = 'serif'
% matplotlib inline

# Other packages we might need
#! pip install quandl
# import quandl

# Options
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
data_folder = '../data/'


height has been deprecated.



In [6]:
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
listings_1 = pd.read_csv(data_folder + 'listings.csv')
# reviews_1 = pd.read_csv(data_folder + 'reviews.csv')
# calendar_1 = pd.read_csv(data_folder + 'calendar.csv')
# neighborhoods_1 = pd.read_csv(data_folder + 'neighbourhoods.csv')

listings = listings_1.copy()

height has been deprecated.



In [9]:
# listings.dtypes
listings.loc[0]
# listings.head(2)

accommodates                                                                   2
amenities                      {"Wireless Internet","Air Conditioning",Kitche...
availability_30                                                                0
bathrooms                                                                      1
bed_type                                                                Real Bed
bedrooms                                                                       1
beds                                                                           1
cancellation_policy                                                     moderate
city                                                                   Asheville
has_availability                                                             NaN
host_id                                                                  2600734
id                                                                        665257
instant_bookable            

In [28]:
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype
# make all numerical data in float
# date_columns = ['first_review', 'last_review', 'calendar_last_scraped', 'host_since', 'last_scraped']
# for col in date_columns:
#     if not is_datetime64_dtype(listings[col]):
#         listings[col] = pd.to_datetime(listings[col])
        
price_columns = ['price', 'weekly_price']
for col in price_columns:
    if not is_numeric_dtype(listings[col]):
        listings[col] = listings[col].str.replace('$', '')
        listings[col] = listings[col].str.replace(',', '')
        listings[col] = listings[col].astype(np.float64)

# percentage_columns = ['host_acceptance_rate']
# for col in percentage_columns:
#     if not is_numeric_dtype(listings[col]):
#         listings[col] = listings[col].str.replace('%', '')
#         listings[col] = listings[col].str.replace(',', '')
#         listings[col] = listings[col].astype(np.float64)/100
        
boolean_columns = ['has_availability','instant_bookable']
for col in boolean_columns:
    for index,value in listings[col].iteritems():
        if listings[col][index]=='t':
            listings.set_value(index, col, 1.0)
        else:
            listings.set_value(index, col, 0.0)
            

In [29]:
# digitize amenities

amenity=listings['amenities'].apply(lambda x: x.replace('"', '').replace('{', '').replace('}', '').split(','))
amenity_dic={}
for index,value in amenity.iteritems():
    for item in value:
        if item != '':
            if item in amenity_dic.keys():
                amenity_dic[item]=amenity_dic[item]+1
            else:
                amenity_dic[item]=1


# print(amenity_dic)
sLength=len(listings['id'])
for key in amenity_dic:
#     print(key)
    listings[key] = pd.Series(np.zeros(sLength), index=listings.index)

for index,value in amenity.iteritems():
    for item in value:
        listings.set_value(index, item, 1.0)



In [31]:
# digitize cancelation policy
sLength=len(listings['id'])
cancelation_policy={}
for index,value in listings['cancellation_policy'].iteritems():
    if value in cancelation_policy.keys():
        cancelation_policy[value]=cancelation_policy[value]+1
    else:
        cancelation_policy[value]=1
        
print(cancelation_policy)
# {'super_strict_60': 2,'super_strict_30': 44,'strict': 2505, 'moderate': 1174,'flexible': 1145,}


listings['cancellation_policy_num'] = pd.Series(np.zeros(sLength), index=listings.index)
cancelation_policy_convert={'no_refunds':0.0,'super_strict_60': 1.0,'super_strict_30': 2.0,'strict': 3.0, 'moderate': 4.0,'flexible': 5.0}

# data=listings['cancellation_policy']
# for index,value in data.iteritems():
#     print(value)
for index,value in listings['cancellation_policy'].iteritems():
#     print(value)
    listings.set_value(index, 'cancellation_policy_num', cancelation_policy_convert[value])



{'moderate': 12570, 'strict': 22277, 'super_strict_30': 27, 'no_refunds': 3, 'super_strict_60': 72, 'flexible': 15372}


In [32]:
# digitize bed type
sLength=len(listings['id'])
bed_type={}
for index,value in listings['bed_type'].iteritems():
    if value in bed_type.keys():
        bed_type[value]=bed_type[value]+1
    else:
        bed_type[value]=1
        
print(bed_type)
# {'super_strict_60': 2,'super_strict_30': 44,'strict': 2505, 'moderate': 1174,'flexible': 1145,}


listings['bed_type_num'] = pd.Series(np.zeros(sLength), index=listings.index)
bed_type_convert={'Real Bed': 0, 'Futon': 1, 'Pull-out Sofa': 2, 'Couch': 3, 'Airbed': 4}

# data=listings['cancellation_policy']
# for index,value in data.iteritems():
#     print(value)
for index,value in listings['bed_type'].iteritems():
#     print(value)
    listings.set_value(index, 'bed_type_num', bed_type_convert[value])




{'Airbed': 309, 'Futon': 471, 'Couch': 189, 'Pull-out Sofa': 290, 'Real Bed': 49062}


In [41]:
# digitize room type
sLength=len(listings['id'])
room_type={}
for index,value in listings['room_type'].iteritems():
    if value in room_type.keys():
        room_type[value]=room_type[value]+1
    else:
        room_type[value]=1
        
print(room_type)
# {'super_strict_60': 2,'super_strict_30': 44,'strict': 2505, 'moderate': 1174,'flexible': 1145,}


listings['room_type_num'] = pd.Series(np.zeros(sLength), index=listings.index)
room_type_convert={'Shared room': 0, 'Private room': 1, 'Entire home/apt': 2}


for index,value in listings['room_type'].iteritems():
#     print(value)
    listings.set_value(index, 'room_type_num', room_type_convert[value])

listings['zipcode'].apply(lambda x: str(x).zfill(5)).dtype

{'Shared room': 1957, 'Entire home/apt': 31586, 'Private room': 16778}


In [42]:
listings.dtypes

accommodates                                  float64
amenities                                      object
availability_30                                 int64
bathrooms                                     float64
bed_type                                       object
bedrooms                                      float64
beds                                          float64
cancellation_policy                            object
city                                           object
has_availability                              float64
host_id                                         int64
id                                              int64
instant_bookable                               object
latitude                                      float64
longitude                                     float64
metropolitan                                   object
name                                           object
price                                         float64
property_type               

In [45]:
listings['zipcode'].apply(lambda x: str(x).zfill(5)).dtypes

dtype('O')

In [43]:
listings.loc[0]

accommodates                                                                                  2
amenities                                     {"Wireless Internet","Air Conditioning",Kitche...
availability_30                                                                               0
bathrooms                                                                                     1
bed_type                                                                               Real Bed
bedrooms                                                                                      1
beds                                                                                          1
cancellation_policy                                                                    moderate
city                                                                                  Asheville
has_availability                                                                              0
host_id                                 